In this notebook we read in the Master XML file made using NapaTrackMater and create N, Delta times Attribute dimensional vectors. N being the number of tracks present in the chosen region, R, Delta being the chosen time interval (t_minus + t_plus) {t - t_minus, t + t_plus} and Attributes being the morphological and the dynamic properties associated with cells in the tracks. We concatenate the Attribute componenets over the chosen time interval to create a Delta times Attribute dimensional vector and create a pandas dataframe with these vectors for further analysis.

In [ ]:
!pip install napatrackmater
!pip install git+https://github.com/kapoorlab/cellshape-helper
!pip install git+https://github.com/kapoorlab/cellshape-cloud
!pip install git+https://github.com/kapoorlab/cellshape-cluster    

In [2]:
%gui qt5
from napatrackmater.Trackvector import TrackVector
from pathlib import Path
from ipywidgets import interactive, widgets
from IPython.display import display
import napari 
from tifffile import imread
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
master_xml_path = Path('C:/Users/rando/Downloads/Mari_project/gt/master_gt_star_hyperstack.xml')
spot_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/spots.csv')
track_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/tracks.csv')
edges_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/edges.csv')
image = imread('C:/Users/rando/Downloads/Mari_project/gt/rawk.tif')
show_tracks = False

shape_dynamic_dict = {'Time':0, 'Track ID':1,  'Dividing':2, 'Number_Dividing':3,  'Radius':4, 'Volume':5,'Eccentricity Comp First':6, 'Eccentricity Comp Second':7, 'Surface Area':8, 'Cluster Class':9, 'Speed':10, 'Directional_Change_Rate':11, 'Acceleration':12}

In [4]:
if show_tracks:
  viewer = napari.Viewer()
else:
    viewer = None
track_vectors = TrackVector(viewer,image,master_xml_path,spot_csv_path, track_csv_path, edges_csv_path, show_tracks = show_tracks)

Reading Master XML
Iterating over spots in frame


TypeError: float() argument must be a string or a real number, not 'NoneType'

In [ ]:
def track_setter(deltat, deltax, deltay):
    track_vectors.t_minus = deltat[0]
    track_vectors.t_plus = deltat[-1]
    
    track_vectors.x_start = deltax[0]
    track_vectors.x_end = deltax[-1]
    
    track_vectors.y_start = deltay[0]
    track_vectors.y_end = deltay[-1]
    

In [ ]:
track_vector_widgets = interactive(track_setter, deltat = widgets.IntRangeSlider(
    value=[track_vectors.tstart, track_vectors.tend],
    min= track_vectors.tstart,
    max=track_vectors.tend,
    step=1,
    description='Delta Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltax = widgets.IntRangeSlider(
    value=[track_vectors.xmin, track_vectors.xmax],
    min= track_vectors.xmin,
    max=track_vectors.xmax,
    step=1,
    description='Delta X',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltay = widgets.IntRangeSlider(
    value=[track_vectors.ymin, track_vectors.ymax],
    min= track_vectors.ymin,
    max=track_vectors.ymax,
    step=1,
    description='Delta Y',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
),                               
                                  
                                  )

track_vector_widgets

In [ ]:
track_vectors._interactive_function()

In [ ]:
current_shape_dynamic_vectors = track_vectors.current_shape_dynamic_vectors

In [ ]:
global_shape_dynamic_dataframe = []
for i in range(len(current_shape_dynamic_vectors)):
   shape_dynamic_dataframe = pd.DataFrame(list(zip(current_shape_dynamic_vectors[i])), index ='Track ID', 
                                                                      columns =['Time', 'Track ID',  'Dividing', 'Number_Dividing',  'Radius', 'Volume', 'Eccentricity Comp First', 'Eccentricity Comp Second', 'Surface Area', 'Cluster Class', 'Speed', 'Directional_Change_Rate', 'Acceleration'])
   if len(global_shape_dynamic_dataframe) == 0:
        global_shape_dynamic_dataframe = shape_dynamic_dataframe
   else:
        global_shape_dynamic_dataframe = pd.concat([global_shape_dynamic_dataframe, shape_dynamic_dataframe],ignore_index=False)


In [ ]:
plt.scatter(np.asarray(number_dividing_property), np.asarray(radius_property))